In [1]:
import pandas as pd
import pyarrow.parquet as pq
import lmdb
import pickle
import os
from tqdm import tqdm
import gc 
import warnings
warnings.filterwarnings('ignore')
from google.cloud import storage
import os
import torch
from google.cloud import bigquery
client = bigquery.Client()
import numpy as np


In [2]:
raw = pd.read_parquet("/home/jupyter/raw.parquet")
raw.shape

(1200000, 23)

In [3]:
raw.head(1)

,clip,syspropertyid,baths_final,state_final,beds_final,property_type_final,pool_final,foundation_final,year_built_final,stories_final,...,addressstreetaddress,addresscity,addressstate,addresspostalcode,publicremarks,train,val,test,just_imagelink1,all_files
0,5594364891,[466][801484][2018][801484],2,IA,3,SFR,no,,1966,-1.0,...,520 SE 6th Ave.,Lemars,IA,51031,A diamond in the rough. This is a solid home i...,False,False,True,GSC_RES_801484_1.jpg,"[GSC_RES_801484_1.jpg, GSC_RES_801484_5.jpg, G..."


In [40]:
df = raw.iloc[10000:12000]
image_dict = df.groupby('just_imagelink1')['all_files'].apply(list).to_dict()

In [41]:
env = lmdb.open('/mnt/disks/multi_mls/database/' + 'mylmdb', map_size=1e12)

def read_lmdb(txn, key):
        lmdb_data = txn.get(key.encode())
        lmdb_data = pickle.loads(lmdb_data)
        return lmdb_data
    
embed_dict = {}
with env.begin(write=False) as txn:
    for key in tqdm(image_dict.keys()):
        # print(read_lmdb(txn, key))
        # break
        try:
          embed = [read_lmdb(txn, i)['embedding'] for i in image_dict[key][0]]
          embed_dict[read_lmdb(txn, key)['publicremarks']] = embed
        except:
          pass

env.close()

100%|██████████| 2000/2000 [00:18<00:00, 105.36it/s]


In [2]:
import lmdb

lmdb_path = '/mnt/disks/multi_mls/database/mylmdb'

# Open the LMDB database
env = lmdb.open(lmdb_path, readonly=True)

# Get the number of keys in the database
with env.begin() as txn:
    num_keys = txn.stat()['entries']

# Close the LMDB database
env.close()

print(f"Number of keys in the LMDB dataset: {num_keys}")

Number of keys in the LMDB dataset: 23741512


In [9]:
import lmdb

lmdb_path = '/mnt/disks/multi_mls/database/mylmdb'

# Open the LMDB database
env = lmdb.open(lmdb_path, readonly=True)

# Get the number of keys in the database and print the first key-value pair
with env.begin() as txn:
    num_keys = txn.stat()['entries']
    first_key = txn.get(b'GSC_RES_801484_1.jpg')  # Assuming keys are byte strings of integers

# Close the LMDB database
env.close()

print(f"Number of keys in the LMDB dataset: {num_keys}\n")
print(pickle.loads(first_key))

Number of keys in the LMDB dataset: 23741512

clip                                                           5594364891
syspropertyid                                 [466][801484][2018][801484]
baths_final                                                             2
state_final                                                            IA
beds_final                                                              3
property_type_final                                                   SFR
pool_final                                                             no
foundation_final                                                         
year_built_final                                                     1966
stories_final                                                        -1.0
square_feet_final                                                  1506.0
fa_listid                                                          801484
fa_apn                                                         121

In [42]:
final_df = pd.DataFrame(embed_dict.items(), columns=['just_imagelink1', 'embeddings'])
final_df.shape

(1788, 2)

In [44]:
final_df.head(1)

,just_imagelink1,embeddings
0,This home just got a face lift. New Paint thro...,"[[0.23979590833187103, 0.6656813025474548, -0...."


In [39]:
final_df.to_pickle('/home/jupyter/open_clip/test_run.pkl')

In [43]:
final_df.to_pickle('/home/jupyter/open_clip/test_run_val.pkl')

In [ ]:
new_query = f"""
SELECT imagelink1,
train, valid, test
FROM `clgx-imanalytics-app-sbx-7342.multi_image_rd.tiny_exploded_complete` 
   """
    
df = client.query(new_query).to_dataframe()
df.shape